This is version 2 of my Fantasy Hockey Analyzer. The purpose of this notebook is to predict the number of fantasy points every hockey player in the league will get based on previous years' performance.

This notebook primarily uses data from moneypuck.com for analysis, and it also uses data from rotowire.com to get +/- for each player.

Section 1: Parameters and Modules

These are the variables that can be adjusted. My model is an ensemble model consisting of neural nets and random forests, with data going back one, two, and three years.

In [1]:
# Set these values to the appropriate ammonts

current_year = 2025
common_number = 0
number_of_one_year_neural_nets = common_number
number_of_two_year_neural_nets = common_number
number_of_three_year_neural_nets = common_number
number_of_one_year_random_forests = common_number
number_of_two_year_random_forests = common_number
number_of_three_year_random_forests = common_number

# This is the breakdown of how many fantasy points a player gets for each category
points_dictionary = {
    'Goals':5, 
    'Assists':3, 
    '+/-':1.5, 
    'PIM':-0.25, 
    'PP_Goals':4, 
    'PP_Assists':2, 
    'SH_Goals':6, #won't count SHG from 5-on-3
    'SH_Assists':4, 
    'Faceoffs_Won':0.25, 
    'Faceoffs_Lost':-0.15, 
    'Hits':0.5, 
    'Blocked_Shots':0.75
    }

The following is a list of modules that I used and the reason why they were used:

-os: to allow the program to read data in the repository

-numpy: basic math operations

-pandas: all dataframe operations/data storage/data cleaning

-various sklearn: all machine learning operations/analysis

In addition to these modules, I also have a custom module that contains helper functions that help in data cleaning/accuracy evaluation. These functions are contained in the "my_module.py" file in the repository. If you are interested in taking a look at these functions, they are available at https://github.com/chrisberry888/FantasyHockeyAnalyzer in the "my_module.py" file.

In [2]:
#Import block
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone
import my_module_v2 as mx

pd.set_option('display.max_columns', None)

Section 2: Data Gathering and Cleaning

This section compiles the Moneypuck and Rotowire data into a format that is usable by the ML models.

There is some discrepencies between stats on ESPN and on moneypuck. These shouldn't alter the fantasy points too much. For example, Sydney Crosby is short-changed 1 faceoff win but has one additional hit in moneypuck than on ESPN, so in my model he has 0.10 more points than he does on ESPN. I would like to see why this is the case in the future/get fully accurate data (perhaps from ESPN themselves), but for now I am ok with this very small error.

In [3]:
yearly_player_data = []

for year in range(2010, current_year):
    moneypuck_data = mx.get_moneypuck_data(year)
    rotowire_data = mx.get_rotowire_data(year)
    combined_data = mx.combine_dataframes(moneypuck_data, rotowire_data)
    final_df = mx.calculate_fantasy_points(combined_data, points_dictionary)
    yearly_player_data.append(final_df)

    

In [4]:
player_id_table = mx.get_player_id_table(yearly_player_data)

In [5]:
player_id_table.head()

,playerId,name
838,8484406,Ty Mueller_VAN_2024_C
839,8484768,Zayne Parekh_CGY_2024_D
856,8485512,Tim Washe_ANA_2024_C
855,8485511,Quinn Hutson_EDM_2024_R
837,8484321,Nikolas Matinpalo_OTT_2024_D


This next cell compiles the yearly data into chunks of one, two, and three-year data to be used by the ML models.

In [6]:
ml_data_one_year = mx.get_ml_data(yearly_player_data, current_year, 1)
ml_data_two_years = mx.get_ml_data(yearly_player_data, current_year, 2)
ml_data_three_years = mx.get_ml_data(yearly_player_data, current_year, 3)

/home/chris/Git_Repositories/FantasyHockeyAnalyzer/my_module_v2.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/home/chris/Git_Repositories/FantasyHockeyAnalyzer/my_module_v2.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/home/chris/Git_Repositories/FantasyHockeyAnalyzer/my_module_v2.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, se

The data is now ready to be used to train the ML model.

In [7]:

one_year_X, one_year_y = mx.separate_fantasy_points(ml_data_one_year)
two_year_X, two_year_y = mx.separate_fantasy_points(ml_data_two_years)
three_year_X, three_year_y = mx.separate_fantasy_points(ml_data_three_years)

In [8]:
ml_data_one_year.head()

,+/-,4on5_I_F_blockedShotAttempts,4on5_I_F_dZoneGiveaways,4on5_I_F_dZoneShiftEnds,4on5_I_F_dZoneShiftStarts,4on5_I_F_faceOffsWon,4on5_I_F_flurryAdjustedxGoals,4on5_I_F_flurryScoreVenueAdjustedxGoals,4on5_I_F_flyShiftEnds,4on5_I_F_flyShiftStarts,4on5_I_F_freeze,4on5_I_F_giveaways,4on5_I_F_goals,4on5_I_F_highDangerGoals,4on5_I_F_highDangerShots,4on5_I_F_highDangerxGoals,4on5_I_F_hits,4on5_I_F_lowDangerGoals,4on5_I_F_lowDangerShots,4on5_I_F_lowDangerxGoals,4on5_I_F_mediumDangerGoals,4on5_I_F_mediumDangerShots,4on5_I_F_mediumDangerxGoals,4on5_I_F_missedShots,4on5_I_F_neutralZoneShiftEnds,4on5_I_F_neutralZoneShiftStarts,4on5_I_F_oZoneShiftEnds,4on5_I_F_oZoneShiftStarts,4on5_I_F_penalityMinutes,4on5_I_F_playContinuedInZone,4on5_I_F_playContinuedOutsideZone,4on5_I_F_playStopped,4on5_I_F_points,4on5_I_F_primaryAssists,4on5_I_F_reboundGoals,4on5_I_F_rebounds,4on5_I_F_reboundxGoals,4on5_I_F_savedShotsOnGoal,4on5_I_F_savedUnblockedShotAttempts,4on5_I_F_scoreAdjustedShotsAttempts,4on5_I_F_scoreAdjustedUnblockedShotAttempts,4on5_I_F_scoreVenueAdjustedxGoals,4on5_I_F_secondaryAssists,4on5_I_F_shifts,4on5_I_F_shotAttempts,4on5_I_F_shotsOnGoal,4on5_I_F_takeaways,4on5_I_F_unblockedShotAttempts,4on5_I_F_xFreeze,4on5_I_F_xGoals,4on5_I_F_xGoalsFromActualReboundsOfShots,4on5_I_F_xGoalsFromxReboundsOfShots,4on5_I_F_xGoals_with_earned_rebounds,4on5_I_F_xGoals_with_earned_rebounds_scoreAdjusted,4on5_I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,4on5_I_F_xOnGoal,4on5_I_F_xPlayContinuedInZone,4on5_I_F_xPlayContinuedOutsideZone,4on5_I_F_xPlayStopped,4on5_I_F_xRebounds,4on5_OffIce_A_shotAttempts,4on5_OffIce_A_xGoals,4on5_OffIce_F_shotAttempts,4on5_OffIce_F_xGoals,4on5_OnIce_A_blockedShotAttempts,4on5_OnIce_A_flurryAdjustedxGoals,4on5_OnIce_A_flurryScoreVenueAdjustedxGoals,4on5_OnIce_A_goals,4on5_OnIce_A_highDangerGoals,4on5_OnIce_A_highDangerShots,4on5_OnIce_A_highDangerxGoals,4on5_OnIce_A_lowDangerGoals,4on5_OnIce_A_lowDangerShots,4on5_OnIce_A_lowDangerxGoals,4on5_OnIce_A_mediumDangerGoals,4on5_OnIce_A_mediumDangerShots,4on5_OnIce_A_mediumDangerxGoals,4on5_OnIce_A_missedShots,4on5_OnIce_A_reboundGoals,4on5_OnIce_A_rebounds,4on5_OnIce_A_reboundxGoals,4on5_OnIce_A_scoreAdjustedShotsAttempts,4on5_OnIce_A_scoreAdjustedUnblockedShotAttempts,4on5_OnIce_A_scoreVenueAdjustedxGoals,4on5_OnIce_A_shotAttempts,4on5_OnIce_A_shotsOnGoal,4on5_OnIce_A_unblockedShotAttempts,4on5_OnIce_A_xGoals,4on5_OnIce_A_xGoalsFromActualReboundsOfShots,4on5_OnIce_A_xGoalsFromxReboundsOfShots,4on5_OnIce_A_xGoals_with_earned_rebounds,4on5_OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,4on5_OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,4on5_OnIce_A_xOnGoal,4on5_OnIce_F_blockedShotAttempts,4on5_OnIce_F_flurryAdjustedxGoals,4on5_OnIce_F_flurryScoreVenueAdjustedxGoals,4on5_OnIce_F_goals,4on5_OnIce_F_highDangerGoals,4on5_OnIce_F_highDangerShots,4on5_OnIce_F_highDangerxGoals,4on5_OnIce_F_lowDangerGoals,4on5_OnIce_F_lowDangerShots,4on5_OnIce_F_lowDangerxGoals,4on5_OnIce_F_mediumDangerGoals,4on5_OnIce_F_mediumDangerShots,4on5_OnIce_F_mediumDangerxGoals,4on5_OnIce_F_missedShots,4on5_OnIce_F_reboundGoals,4on5_OnIce_F_rebounds,4on5_OnIce_F_reboundxGoals,4on5_OnIce_F_scoreAdjustedShotsAttempts,4on5_OnIce_F_scoreAdjustedUnblockedShotAttempts,4on5_OnIce_F_scoreVenueAdjustedxGoals,4on5_OnIce_F_shotAttempts,4on5_OnIce_F_shotsOnGoal,4on5_OnIce_F_unblockedShotAttempts,4on5_OnIce_F_xGoals,4on5_OnIce_F_xGoalsFromActualReboundsOfShots,4on5_OnIce_F_xGoalsFromxReboundsOfShots,4on5_OnIce_F_xGoals_with_earned_rebounds,4on5_OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,4on5_OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,4on5_OnIce_F_xOnGoal,4on5_corsiAgainstAfterShifts,4on5_corsiForAfterShifts,4on5_faceoffsLost,4on5_faceoffsWon,4on5_fenwickAgainstAfterShifts,4on5_fenwickForAfterShifts,4on5_gameScore,4on5_iceTimeRank,4on5_icetime,4on5_offIce_corsiPercentage,4on5_offIce_fenwickPercentage,4on5_offIce_xGoalsPercentage,4on5_onIce_corsiPercentage,4on5_onIce_fenwickPercentage,4on5_onIce_

In [22]:
one_year_neural_nets = mx.create_models(
    one_year_X,
    one_year_y,
    MLPRegressor(max_iter=1000),
    10 #number_of_one_year_neural_nets
)

In [10]:
one_year_random_forests = mx.create_models(
    one_year_X,
    one_year_y,
    RandomForestRegressor(),
    number_of_one_year_random_forests
)

In [11]:
two_year_neural_nets = mx.create_models(
    two_year_X,
    two_year_y,
    MLPRegressor(max_iter=1000),
    number_of_two_year_neural_nets
)

In [12]:
two_year_random_forests = mx.create_models(
    two_year_X,
    two_year_y,
    RandomForestRegressor(),
    number_of_two_year_random_forests
)

In [13]:
three_year_neural_nets = mx.create_models(
    three_year_X,
    three_year_y,
    MLPRegressor(max_iter=1000),
    number_of_three_year_neural_nets
)

In [14]:
three_year_random_forests = mx.create_models(
    three_year_X,
    three_year_y,
    RandomForestRegressor(),
    number_of_three_year_random_forests
)

Now we generate the table with final predictions.

In [15]:
current_one_year_X = mx.get_final_year_data(yearly_player_data, 1)
current_two_year_X = mx.get_final_year_data(yearly_player_data, 2)
current_three_year_X = mx.get_final_year_data(yearly_player_data, 3)

In [16]:

display(current_one_year_X.head())

,+/-,4on5_I_F_blockedShotAttempts,4on5_I_F_dZoneGiveaways,4on5_I_F_dZoneShiftEnds,4on5_I_F_dZoneShiftStarts,4on5_I_F_faceOffsWon,4on5_I_F_flurryAdjustedxGoals,4on5_I_F_flurryScoreVenueAdjustedxGoals,4on5_I_F_flyShiftEnds,4on5_I_F_flyShiftStarts,4on5_I_F_freeze,4on5_I_F_giveaways,4on5_I_F_goals,4on5_I_F_highDangerGoals,4on5_I_F_highDangerShots,4on5_I_F_highDangerxGoals,4on5_I_F_hits,4on5_I_F_lowDangerGoals,4on5_I_F_lowDangerShots,4on5_I_F_lowDangerxGoals,4on5_I_F_mediumDangerGoals,4on5_I_F_mediumDangerShots,4on5_I_F_mediumDangerxGoals,4on5_I_F_missedShots,4on5_I_F_neutralZoneShiftEnds,4on5_I_F_neutralZoneShiftStarts,4on5_I_F_oZoneShiftEnds,4on5_I_F_oZoneShiftStarts,4on5_I_F_penalityMinutes,4on5_I_F_playContinuedInZone,4on5_I_F_playContinuedOutsideZone,4on5_I_F_playStopped,4on5_I_F_points,4on5_I_F_primaryAssists,4on5_I_F_reboundGoals,4on5_I_F_rebounds,4on5_I_F_reboundxGoals,4on5_I_F_savedShotsOnGoal,4on5_I_F_savedUnblockedShotAttempts,4on5_I_F_scoreAdjustedShotsAttempts,4on5_I_F_scoreAdjustedUnblockedShotAttempts,4on5_I_F_scoreVenueAdjustedxGoals,4on5_I_F_secondaryAssists,4on5_I_F_shifts,4on5_I_F_shotAttempts,4on5_I_F_shotsOnGoal,4on5_I_F_takeaways,4on5_I_F_unblockedShotAttempts,4on5_I_F_xFreeze,4on5_I_F_xGoals,4on5_I_F_xGoalsFromActualReboundsOfShots,4on5_I_F_xGoalsFromxReboundsOfShots,4on5_I_F_xGoals_with_earned_rebounds,4on5_I_F_xGoals_with_earned_rebounds_scoreAdjusted,4on5_I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,4on5_I_F_xOnGoal,4on5_I_F_xPlayContinuedInZone,4on5_I_F_xPlayContinuedOutsideZone,4on5_I_F_xPlayStopped,4on5_I_F_xRebounds,4on5_OffIce_A_shotAttempts,4on5_OffIce_A_xGoals,4on5_OffIce_F_shotAttempts,4on5_OffIce_F_xGoals,4on5_OnIce_A_blockedShotAttempts,4on5_OnIce_A_flurryAdjustedxGoals,4on5_OnIce_A_flurryScoreVenueAdjustedxGoals,4on5_OnIce_A_goals,4on5_OnIce_A_highDangerGoals,4on5_OnIce_A_highDangerShots,4on5_OnIce_A_highDangerxGoals,4on5_OnIce_A_lowDangerGoals,4on5_OnIce_A_lowDangerShots,4on5_OnIce_A_lowDangerxGoals,4on5_OnIce_A_mediumDangerGoals,4on5_OnIce_A_mediumDangerShots,4on5_OnIce_A_mediumDangerxGoals,4on5_OnIce_A_missedShots,4on5_OnIce_A_reboundGoals,4on5_OnIce_A_rebounds,4on5_OnIce_A_reboundxGoals,4on5_OnIce_A_scoreAdjustedShotsAttempts,4on5_OnIce_A_scoreAdjustedUnblockedShotAttempts,4on5_OnIce_A_scoreVenueAdjustedxGoals,4on5_OnIce_A_shotAttempts,4on5_OnIce_A_shotsOnGoal,4on5_OnIce_A_unblockedShotAttempts,4on5_OnIce_A_xGoals,4on5_OnIce_A_xGoalsFromActualReboundsOfShots,4on5_OnIce_A_xGoalsFromxReboundsOfShots,4on5_OnIce_A_xGoals_with_earned_rebounds,4on5_OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,4on5_OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,4on5_OnIce_A_xOnGoal,4on5_OnIce_F_blockedShotAttempts,4on5_OnIce_F_flurryAdjustedxGoals,4on5_OnIce_F_flurryScoreVenueAdjustedxGoals,4on5_OnIce_F_goals,4on5_OnIce_F_highDangerGoals,4on5_OnIce_F_highDangerShots,4on5_OnIce_F_highDangerxGoals,4on5_OnIce_F_lowDangerGoals,4on5_OnIce_F_lowDangerShots,4on5_OnIce_F_lowDangerxGoals,4on5_OnIce_F_mediumDangerGoals,4on5_OnIce_F_mediumDangerShots,4on5_OnIce_F_mediumDangerxGoals,4on5_OnIce_F_missedShots,4on5_OnIce_F_reboundGoals,4on5_OnIce_F_rebounds,4on5_OnIce_F_reboundxGoals,4on5_OnIce_F_scoreAdjustedShotsAttempts,4on5_OnIce_F_scoreAdjustedUnblockedShotAttempts,4on5_OnIce_F_scoreVenueAdjustedxGoals,4on5_OnIce_F_shotAttempts,4on5_OnIce_F_shotsOnGoal,4on5_OnIce_F_unblockedShotAttempts,4on5_OnIce_F_xGoals,4on5_OnIce_F_xGoalsFromActualReboundsOfShots,4on5_OnIce_F_xGoalsFromxReboundsOfShots,4on5_OnIce_F_xGoals_with_earned_rebounds,4on5_OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,4on5_OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,4on5_OnIce_F_xOnGoal,4on5_corsiAgainstAfterShifts,4on5_corsiForAfterShifts,4on5_faceoffsLost,4on5_faceoffsWon,4on5_fenwickAgainstAfterShifts,4on5_fenwickForAfterShifts,4on5_gameScore,4on5_iceTimeRank,4on5_icetime,4on5_offIce_corsiPercentage,4on5_offIce_fenwickPercentage,4on5_offIce_xGoalsPercentage,4on5_onIce_corsiPercentage,4on5_onIce_fenwickPercentage,4on5_onIce_

In [20]:
one_year_neural_net_predictions = mx.get_prediction_table(one_year_neural_nets, current_one_year_X, player_id_table)


In [21]:
one_year_neural_net_predictions.head()

,playerId,name,prediction
587,8477492,Nathan MacKinnon_COL_2024_C,633.566159
827,8471675,Sidney Crosby_PIT_2024_C,601.783200
359,8480002,Nico Hischier_NJD_2024_C,577.496699
590,8477933,Sam Reinhart_FLA_2024_C,567.431680
491,8478427,Sebastian Aho_CAR_2024_C,550.348210
